In [1]:
import os
import numpy as np
from numpy.linalg import norm
import trimesh
import pybullet as p
from scipy.spatial.transform import Rotation as R

# cross can be 0
np.random.seed(32)
ASSET_ROOT = os.path.abspath('urdfc')

cube = trimesh.load(ASSET_ROOT+'/cube/tinker.obj')
cuboid = trimesh.load(ASSET_ROOT+'/cuboid/tinker.obj')
cyl = trimesh.load(ASSET_ROOT+'/cylinder/tinker.obj')
cc = trimesh.load(ASSET_ROOT+'/cut_cuboid/tinker.obj')
sc = trimesh.load(ASSET_ROOT+'/small_cuboid/tinker.obj')
tc = trimesh.load(ASSET_ROOT+'/thin_cuboid/tinker.obj')
roof = trimesh.load(ASSET_ROOT+'/roof/tinker.obj')
pyr = trimesh.load(ASSET_ROOT+'/pyramid/tinker.obj')

obj_primes = [cube, cyl, cc, sc, tc, roof, pyr, cuboid]
obj_vertcs = set([len(x.vertices) for x in obj_primes])
# [8, 40, 74, 8, 8, 66, 6, 8]
irg_vertc_map = {
    40: 'cylinder',
    74: 'cut_cuboid',
    66: 'roof',
    6: 'pyramid',
    8: '[regular]'
}

for x in obj_primes:
    x.apply_scale(0.001)

def get_transform(rotq=None, euler=None, rotvec=None, matrix=None, pos=(0,0,0)):
    trans = np.eye(4)
    
    if rotq is not None:
        trans[:-1,:-1] = R.from_quat(rotq).as_matrix()  
    elif euler is not None:
        trans[:-1,:-1] = R.from_euler('xyz', euler).as_matrix()
    elif rotvec is not None:
        trans[:-1,:-1] = R.from_rotvec(rotvec).as_matrix()
    elif matrix is not None:
        trans[:-1,:-1] = matrix
        
    trans[:-1,-1:] = np.array(pos).reshape(-1,1)
    
    return trans

def mesh_center(mesh):
    vert_count = len(mesh.vertices)
    center = None
    assert(vert_count in obj_vertcs)
    
    if irg_vertc_map[vert_count] == 'cut_cuboid':
        # cm to nearest
        cm = mesh.center_mass
        (center,),*_ = trimesh.proximity.closest_point(mesh, [cm])
        
    elif irg_vertc_map[vert_count] in ['pyramid', 'roof']:
        # xy from cm, heighest z value/2
        vert_zs = mesh.vertices[:,-1]
        hi_z = vert_zs.max()
        lo_z = vert_zs.min()
        
        mid_z = (hi_z+lo_z)/2
        
        new_x, new_y, _ = mesh.center_mass
        center = (new_x, new_y, mid_z)
    else:
        center = mesh.center_mass

    return np.array(center)
        
def to_origin(mesh):
    new_offt = mesh_center(mesh)
    mesh.apply_transform(get_transform(pos=-new_offt))
    
for x in obj_primes:
    to_origin(x)

omap = {
    'cube': cube,
    'cylinder': cyl,
    'cut_cuboid': cc,
    'small_cuboid': sc,
    'thin_cuboid': tc,
    'roof': roof,
    'pyramid': pyr,
    'cuboid': cuboid
}
assert(len(omap) == len(obj_primes))
def make_obj(otype, om=omap):
    return om[otype].copy()

pybullet build time: Nov 30 2022 16:56:59


In [2]:
def get_path(vpath, mesh):
    return trimesh.load_path(mesh.vertices[vpath])

def recreate(objs):
    oid_to_mesh = {}
    
    for oid, oty in objs:
        mesh = make_obj(oty)
        
        m_pos, m_ori = p.getBasePositionAndOrientation(oid)
        mesh.apply_transform(get_transform(rotq=m_ori, pos=m_pos))
        
        oid_to_mesh[oid] = mesh
        
    sc = trimesh.Scene(list(oid_to_mesh.values()))
        
    return bidict(oid_to_mesh), sc

In [3]:
import os
import pybullet as p
import pybullet_data
from numpy import pi
from bidict import bidict
import time

class PBObjectLoader:
    def __init__(self, asset_root):
        self.obj_ids = []
        self.obj_typ = []
        self.asset_root = os.path.abspath(asset_root)
        
    def load_obj(self, otype, pos=(0,0,0), ori=(0,0,0,1), wait=0):
        if isinstance(ori, list):
            ori = p.getQuaternionFromEuler(ori)
        
        oid = p.loadURDF(os.path.join(self.asset_root, f'{otype}.urdf'), pos, ori)

        for _ in range(wait):
            p.stepSimulation()
            time.sleep(1./240.)
            
        self.obj_ids.append(oid)
        self.obj_typ.append(otype)
            
        return oid
    
    def get_objs(self):
        return list(zip(self.obj_ids, self.obj_typ))

PLANE_ROOT = os.path.abspath('urdf')

physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)

target = (-0.07796166092157364, 0.005451506469398737, -0.06238798052072525)
dist = 1.0
yaw = 89.6000747680664
pitch = -17.800016403198242
p.resetDebugVisualizerCamera(dist,yaw,pitch,target)

planeId = p.loadURDF(os.path.join(PLANE_ROOT, 'plane/plane.urdf'))

loader = PBObjectLoader('urdfc')

loader.load_obj('cut_cuboid', pos=[0.05, 0, 0.01], ori=[0, 0, 0])
loader.load_obj('cube', pos=[0, 0, 0.01], ori=[0, 0, 0], wait=100)
# loader.load_obj('cube', pos=[0.025, 0, .1], ori=[pi/2, pi/2, 0], wait=100)

om_map, sc = recreate(loader.get_objs())

time.sleep(1)
p.disconnect()

sc.show()
scOrig = sc.copy()

In [4]:
np.array([[(0,0,1),(1,0,0),(0,1,0)],[(1,2,3),(1,2,3),(1,2,3)]]).mean(axis=1).mean(axis=0)

array([0.66666667, 1.16666667, 1.66666667])

In [5]:
xt = np.array([(1,2,3),(3,4,5),(1,2,3)])
np.unique(xt, axis=0)

array([[1, 2, 3],
       [3, 4, 5]])

In [11]:
import numpy as np
from numpy.linalg import norm
from scipy.spatial.distance import cdist
from numpy.linalg import norm
from scipy.spatial.transform import Rotation as R
from scipy.linalg import lstsq
from scipy.optimize import least_squares

def top_faces_idx(mesh):
    facet_n = mesh.facets_normal
    up_n = facet_n[facet_n[:,-1].argmax()]
    face_n = mesh.face_normals
    
    return np.all(np.isclose(face_n, up_n), axis=1).nonzero()

def bottom_faces_fcentroid(mesh):
    facet_n = mesh.facets_normal
    down_n = facet_n[facet_n[:,-1].argmin()]
    face_n = mesh.face_normals
    
    f_idx = np.all(np.isclose(face_n, down_n), axis=1)
    b_faces = mesh.faces[f_idx]
    bf_vert = mesh.vertices[b_faces]
    bf_cens = bf_vert.mean(axis=1)
    
    return bf_cens.mean(axis=0)

def closest_point(set1, set2):
    dists = cdist(set1, set2)
    min_i, min_j = np.unravel_index(dists.argmin(), dists.shape)
    
    return min_i, min_j

def closest_points_idx(set1, set2, n):
    dists = cdist(set1, set2)
    indices = dists.flatten().argsort()[:n]
    indices = np.unravel_index(indices, dists.shape)
    
    return indices

def edges_midpoints(edges):
    return edges.sum(axis=1)/2

def sample_infaces(mesh, faces_idx, num_points):
    face_areas = mesh.area_faces[faces_idx]
    face_probs = face_areas/np.sum(face_areas)
    
    ar_w = np.zeros_like(mesh.area_faces)
    ar_w[faces_idx] = mesh.area_faces[faces_idx]

    points, faces_idx = mesh.sample(num_points, return_index=True, face_weight=ar_w)
    
    return points, faces_idx

def fit_plane(points):
    centroid = points.mean(axis=0)
    _, values, vectors = np.linalg.svd(points - centroid)
    normal = vectors[2]
    d = -np.dot(centroid, normal)
    plane = np.append(normal, d)

    return plane

def orn_from_plane(plane, quat=False):
    '''
    theta = acos(dot(n_left,n_right)/(norm(n_left)*norm(n_right)));
    theta = 1.3876;
    axis = cross(n_left,n_right) / norm(cross(n_left,n_right));
    axis = (-0.0062;-1.0000;0.0053);
    C_matrix = [0     -axis(3) axis(2); 
                axis(3)  0     -axis(1);  
                -axis(2) axis(1)  0];  %cross product matrix
    R = diag([1 1 1]) + (1-cos(theta))*C_matrix^2 + sin(theta)*C_matrix;
    R = [0.1823,-0.0001,-0.9833;
        0.0104,0.9999,0.0018;
        0.9832,-0.0105,0.1822];
    after_rotation = R*blue_points;
    '''
    *normal,_ = plane
    u = normal/norm(normal)
    
    denom = np.sqrt((u*u).sum())
    orn = np.arcsin(u/denom)
    
    if quat:
        orn = R.from_euler(rot, 'xyz').as_quat()
    
    return orn

def get_place_plane(mesh1, mesh2, sim_tol=1e-01, n_closest=10, n_samples=100):
    m1_maxh = mesh1.vertices[:,:-1].max()
    m2_maxh = mesh2.vertices[:,:-1].max()
    
    m1_topf_idx = top_faces_idx(mesh1)
    m2_topf_idx = top_faces_idx(mesh2)
    
    heights_sim = abs(m1_maxh - m2_maxh) < sim_tol
    place_plane = None
    
    m1p, m1pf = sample_infaces(mesh1, m1_topf_idx, n_samples)
    m2p, m2pf = sample_infaces(mesh2, m2_topf_idx, n_samples)
    
    cp_1, cp_2 = closest_points_idx(m1p, m2p, n_closest)
    plane_points = np.concatenate([m1p[cp_1], m2p[cp_2]])
        
    # weighted sum
    pp_f = np.append(m1pf[cp_1], m2pf[cp_2])
    pp_f = pp_f == pp_f[0]
    print(pp_f.shape)
    print(plane_points.shape)
    pp_cen = (plane_points[pp_f].mean(axis=0)+plane_points[~pp_f].mean(axis=0))/2
    
    pp_eqn = fit_plane(plane_points)
    
    return pp_eqn, pp_cen, plane_points

o1, o2 = loader.obj_ids

def get_align_transform(p_mesh, pp_eqn, pp_cen):
    bf_cen = bottom_faces_fcentroid(p_mesh)
    bfc_to_ori = get_transform(pos=-bf_cen)
    
    *normal,_ = pp_eqn
    u = normal/norm(normal)
    rotv = np.cross([0, 0, 1], u)
    
    plane_rot = np.eye(4)
    plane_t = np.eye(4)
    
#     plane_rot = get_transform(rotvec=rotv)
#     plane_t = get_transform(pos=pp_cen)
    plane_rot = get_transform(rotvec=rotv, pos=pp_cen)
#     plane_rot = np.eye(4)
    
    return plane_t@plane_rot@bfc_to_ori

col1, col2 = [om_map[x] for x in loader.obj_ids]

def pspheres(points, radius=.0005):
    return [trimesh.primitives.Sphere(radius=radius, center=pt) for pt in points]

# get plane for placement
pp_eqn, pp_cen, pp_points = get_place_plane(col1, col2)

# decide which object to place
p_obj = cube.copy()

# get transformation to place object
align_trans = get_align_transform(p_obj, pp_eqn, pp_cen)

# place object on plane
p_obj.apply_transform(align_trans)

sc1 = scOrig.copy()
# sc1 = trimesh.Scene()
sc1.add_geometry(pspheres(pp_points))
sc1.add_geometry(pspheres([pp_cen]))
# sc1.add_geometry([p_obj])

sc1.add_geometry([pspheres([(0,0,0)], radius=0.01)])


sc1.show()

(20,)
(20, 3)


In [ ]:
import scipy

scipy.__version__

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# create x,y
x = np.linspace(min(pp_points[:,0]), max(pp_points[:,0]), 100)
y = np.linspace(min(pp_points[:,1]), max(pp_points[:,1]), 100)
xx, yy = np.meshgrid(x, y)

# calculate corresponding z
a,b,c,d = pp_eqn
z = (-a * xx - b * yy - d) * 1./c

# plot the surface
fig = plt.figure()

# Add an axes
ax = fig.add_subplot(111,projection='3d')

# plot the surface
ax.plot_surface(xx, yy, z, alpha=0.2)

# and plot the point 
ax.scatter(pp_points[:,0], pp_points[:,1], pp_points[:,2],  color='green')